In [16]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [1]:
import sqlite3
import pandas as pd


conn = sqlite3.connect('Database/S&P500.db')
query = "SELECT * FROM stock"
# stock = pd.read_sql_query(query, conn)
stock = pd.read_csv('Data/Stock/S&P400.csv')
ticker_by_sector = stock.groupby(by='gics_sector')['symbol'].apply(list).to_dict()

conn.close()

In [12]:
import pandas as pd


def expand_to_2minutes(row, columns):
    dates = pd.date_range(
        start=row.name + pd.Timedelta(hours=14, minutes=30),
        end=row.name + pd.Timedelta(hours=20, minutes=55),
        freq='2min'
    )
    expanded_rows = pd.DataFrame(index=dates, columns=columns)
    expanded_rows.index.name = 'Date'
    for col in columns:
        expanded_rows[col] = row[col]
    return expanded_rows

In [37]:
for sector, tickers in ticker_by_sector.items():
    for ticker in tickers:
        print(ticker)

FYBR
IRDM
NXST
NYT
TKO
WMG
ZI
ALV
AN
ANF
ARMK
BC
BLD
BURL
BYD
CHDN
CHH
CHWY
COLM
CPRI
CRI
CROX
DKS
DUOL
FIVE
FND
GAP
GHC
GME
GNTX
GT
H
HGV
HOG
HRB
JWN
KBH
LAD
LEA
LNW
LOPE
M
MAT
MTN
MUSA
OLLI
PAG
PII
PLNT
PVH
RH
SCI
SKX
THO
TMHC
TNL
TOL
TPX
TXRH
UA
UAA
VAC
VC
VVV
WEN
WH
WHR
WING
WSM
YETI
BJ
BRBR
CASY
CELH
COKE
COTY
DAR
ELF
FLO
INGR
LANC
PFGC
POST
PPC
SAM
SFM
USFD
AM
AR
CHRD
CHX
CIVI
CNX
DINO
DTM
EXE
MTDR
MUR
NOV
OVV
PBF
PR
RRC
VAL
VNOM
WFRD
AFG
ALLY
AMG
ASB
BHF
CADE
CBSH
CFR
CG
CNO
COLB
EEFT
EQH
ESNT
EVR
EWBC
FAF
FCFS
FFIN
FHI
FHN
FNB
FNF
FOUR
GBCI
HLI
HLNE
HOMB
HWC
IBKR
IBOC
JEF
JHG
KMPR
KNSL
MORN
MTG
NLY
NYCB
ONB
ORI
OZK
PB
PNFP
PRI
RGA
RLI
RNR
RYAN
SEIC
SF
SIGI
SLM
SNV
SSB
STWD
TCBI
THG
UBSI
UMBF
UNM
VLY
VOYA
WAL
WBS
WEX
WTFC
WU
ZION
ACHC
AMED
ARWR
AVTR
BIO
BMRN
BRKR
CHE
CYTK
DOCS
EHC
ENOV
ENSG
EXEL
GMED
HAE
HALO
HQY
ILMN
JAZZ
LIVN
LNTH
MASI
MEDP
NBIX
NEOG
NVST
OPCH
PEN
PRGO
RGEN
ROIV
SHC
SRPT
THC
UTHR
XRAY
AAL
AAON
ACM
AGCO
AIT
AYI
BCO
BWXT
CACI
CAR
CLH
CNH
CNM
CNXC
CR
CSL
CW
DCI
E

In [23]:
import yfinance as yf


for sector, tickers in ticker_by_sector.items():
    for ticker in tickers:
        if ticker != 'OHI':
            continue
        '''
        일일 데이터 불러온 후 2분 단위 데이터로 확장
        '''
        start = pd.to_datetime('now') - pd.Timedelta(days=440)
        end = pd.to_datetime('now') - pd.Timedelta(days=59)

        daily_data = yf.download(tickers=ticker, start=f"{start.year}-{start.month}-{start.day}", end=f"{end.year}-{end.month}-{end.day}", interval='1d')
        if daily_data.empty:
            continue
        
        expanded_data = pd.concat([expand_to_2minutes(row, daily_data.columns) for _, row in daily_data.iterrows()])
        print(start, end)
        
        
        '''
        2분 단위 데이터 불러오기
        '''
        start = pd.to_datetime('now') - pd.Timedelta(days=59)
        end = pd.to_datetime('now')
        
        intraday_data = yf.download(tickers=ticker, start=f"{start.year}-{start.month}-{start.day}", end=f"{end.year}-{end.month}-{end.day}", interval='2m')
        if intraday_data.empty:
            continue
        
        intraday_data = intraday_data.rename_axis('Date')
        intraday_data.index = intraday_data.index.tz_localize(None)
        print(start, end)
        
        
        display(intraday_data.head())

[*********************100%***********************]  1 of 1 completed


2023-10-20 00:00:10.570659 2024-11-04 00:00:10.570893


[*********************100%***********************]  1 of 1 completed

2024-11-04 00:00:11.382758 2025-01-02 00:00:11.382920


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,OHI,OHI,OHI,OHI,OHI,OHI
Date,,,,,,
2024-11-14 14:30:00,40.115002,40.115002,40.200001,40.000000,40.000000,28725
2024-11-14 14:34:00,40.189999,40.189999,40.310001,40.189999,40.235001,9436
2024-11-14 14:38:00,40.009998,40.009998,40.110001,40.009998,40.105000,12029
2024-11-14 14:40:00,40.025002,40.025002,40.025002,40.000000,40.000000,2482
2024-11-14 14:42:00,40.000000,40.000000,40.020000,39.980000,40.000000,1037


In [39]:
tmp = yf.download(tickers="AAPL", start=f"2024-11-04", end=f"2024-11-14", interval='2m')

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAPL']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (2m 2024-11-04 -> 2024-11-14)')


In [33]:
display(tmp.head())

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,OHI,OHI,OHI,OHI,OHI,OHI
Datetime,,,,,,
2024-11-14 14:30:00+00:00,40.115002,40.115002,40.200001,40.000000,40.000000,28725
2024-11-14 14:34:00+00:00,40.189999,40.189999,40.310001,40.189999,40.235001,9436
2024-11-14 14:38:00+00:00,40.009998,40.009998,40.110001,40.009998,40.105000,12029
2024-11-14 14:40:00+00:00,40.025002,40.025002,40.025002,40.000000,40.000000,2482
2024-11-14 14:42:00+00:00,40.000000,40.000000,40.020000,39.980000,40.000000,1037


In [22]:
display(expanded_data.tail())
display(intraday_data.head())

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,OHI,OHI,OHI,OHI,OHI,OHI
Date,,,,,,
2024-11-01 20:46:00,40.530003,41.200001,42.59,40.959999,42.509998,1901000.0
2024-11-01 20:48:00,40.530003,41.200001,42.59,40.959999,42.509998,1901000.0
2024-11-01 20:50:00,40.530003,41.200001,42.59,40.959999,42.509998,1901000.0
2024-11-01 20:52:00,40.530003,41.200001,42.59,40.959999,42.509998,1901000.0
2024-11-01 20:54:00,40.530003,41.200001,42.59,40.959999,42.509998,1901000.0


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,OHI,OHI,OHI,OHI,OHI,OHI
Date,,,,,,
2024-11-14 14:30:00,40.115002,40.115002,40.200001,40.000000,40.000000,28725
2024-11-14 14:34:00,40.189999,40.189999,40.310001,40.189999,40.235001,9436
2024-11-14 14:38:00,40.009998,40.009998,40.110001,40.009998,40.105000,12029
2024-11-14 14:40:00,40.025002,40.025002,40.025002,40.000000,40.000000,2482
2024-11-14 14:42:00,40.000000,40.000000,40.020000,39.980000,40.000000,1037


In [24]:
intraday_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5989 entries, 2024-11-14 14:30:00 to 2024-12-31 20:58:00
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   (Adj Close, OHI)  5989 non-null   float64
 1   (Close, OHI)      5989 non-null   float64
 2   (High, OHI)       5989 non-null   float64
 3   (Low, OHI)        5989 non-null   float64
 4   (Open, OHI)       5989 non-null   float64
 5   (Volume, OHI)     5989 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 327.5 KB
